In [50]:
import pandas as pd 
import numpy as np 

- Read Data from Source 1 (State, Capital, StateID)

In [51]:
state=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states.json',dtype=str)
state=state[state['name']!='LUAR NEGARA']
state

,name,name_long,id,code2,code3,capital
0,JOHOR,JOHOR DARUL TAKZIM,1,JH,JOH,JOHOR BAHRU
1,KEDAH,KEDAH DARUL AMAN,2,KD,KED,ALOR SETAR
2,KELANTAN,KELANTAN DARUL NAIM,3,KT,KEL,KOTA BAHRU
3,MELAKA,MELAKA BANDARAYA BERSEJARAH,4,ML,MLK,BANDAR MELAKA
4,NEGERI SEMBILAN,NEGERI SEMBILAN DARUL KHUSUS,5,NS,NSN,SEREMBAN
5,PAHANG,PAHANG DARUL MAKMUR,6,PH,PAH,KUANTAN
6,PULAU PINANG,PULAU PINANG PULAU MUTIARA,7,PN,PEN,GEORGE TOWN
7,PERAK,PERAK DARUL RIDZUAN,8,PR,PRK,IPOH
8,PERLIS,PERLIS INDERA KAYANGAN,9,PL,PER,KANGAR
9,SELANGOR,SELANGOR DARUL EHSAN,10,SG,SEL,SHAH ALAM


- Read Data from Source 2 (StateID, District, DistrictID)

In [52]:
district=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/district.json',dtype=str)
district=district[district['state_id']!='nan'].drop(columns='//comment')
district

,id,state_id,name,code
1,900,9,PERLIS,PER
2,201,2,KOTA SETAR,KSE
3,202,2,KUBANG PASU,KPA
4,203,2,PADANG TERAP,PTE
5,204,2,LANGKAWI,LAN
...,...,...,...,...
122,1307,13,KAPIT,KAP
123,1308,13,SAMARAHAN,SAM
124,1309,13,BINTULU,BIN
125,1310,13,MUKAH,MUK


- Read Data from Source 3 (SubDistrict, SubDistrictID, DistrictID)

In [53]:
mukim=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/mukim.json',dtype=str)
mukim=mukim[mukim['name']!='-']#.drop(columns='//comment')
mukim

,id,name,district_id
0,090001,ABI,900
1,090002,ARAU,900
2,090003,BERSERI,900
3,090004,CHUPING,900
4,090005,UTAN AJI,900
...,...,...,...
1985,130947,ANAP,1309
1986,130948,KAKUS,1309
1987,130954,JELALONG,1309
1988,130955,BINIA,1309


- Left Join (mukim,district) on (DistrictID,ID)

In [54]:
mukim_district=\
pd.merge(
    mukim.drop(columns='id').rename(columns={'name': 'mukim'}),
    district.drop(columns='code').rename(columns={'name': 'district'}),
    how='left',
    left_on='district_id',
    right_on='id'
)

mukim_district

,mukim,district_id,id,state_id,district
0,ABI,900,900,9,PERLIS
1,ARAU,900,900,9,PERLIS
2,BERSERI,900,900,9,PERLIS
3,CHUPING,900,900,9,PERLIS
4,UTAN AJI,900,900,9,PERLIS
...,...,...,...,...,...
1985,ANAP,1309,1309,13,BINTULU
1986,KAKUS,1309,1309,13,BINTULU
1987,JELALONG,1309,1309,13,BINTULU
1988,BINIA,1309,1309,13,BINTULU


- Left Join (mukim_district, state) on (StateID, ID)

In [55]:
mukim_district_state=pd.merge(mukim_district.drop(columns=['district_id','id']),state,how='left',left_on='state_id',right_on='id').drop(columns=['name_long','capital','id','code2','code3']).rename(columns={'name': 'state'})
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


- Identify Mukim with Null State

In [56]:
mukim_district_state[mukim_district_state['state'].isnull()]

,mukim,state_id,district,state
1294,AMPANG,NaN,NaN,NaN
1295,BATU,NaN,NaN,NaN
1296,CHERAS,NaN,NaN,NaN
1297,ULU KELANG,NaN,NaN,NaN
1298,KUALA LUMPUR,NaN,NaN,NaN
1299,PETALING,NaN,NaN,NaN
1300,SETAPAK,NaN,NaN,NaN
1301,BANDAR KUALA LUMPUR,NaN,NaN,NaN
1302,BANDAR PETALING JAYA,NaN,NaN,NaN
1303,BANDAR BANDAR BAHARU SUNGAI BESI,NaN,NaN,NaN


In [57]:
mukim_district_state[mukim_district_state['state']=='PULAU PINANG']

,mukim,state_id,district,state
666,MUKIM 1,7,SEBERANG PERAI TENGAH,PULAU PINANG
667,MUKIM 2,7,SEBERANG PERAI TENGAH,PULAU PINANG
668,MUKIM 3,7,SEBERANG PERAI TENGAH,PULAU PINANG
669,MUKIM 4,7,SEBERANG PERAI TENGAH,PULAU PINANG
670,MUKIM 5,7,SEBERANG PERAI TENGAH,PULAU PINANG
...,...,...,...,...
758,MUKIM H,7,BARAT DAYA,PULAU PINANG
759,MUKIM I,7,BARAT DAYA,PULAU PINANG
760,MUKIM J,7,BARAT DAYA,PULAU PINANG
761,BANDAR BALIK PULAU,7,BARAT DAYA,PULAU PINANG


- Fill in Null with Logical Value

In [58]:
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='AMPANG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BATU','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='CHERAS','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='ULU KELANG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='KUALA LUMPUR','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PETALING','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='SETAPAK','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR KUALA LUMPUR','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR PETALING JAYA','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='BANDAR BANDAR BAHARU SUNGAI BESI','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN BATU','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN BATU CAVES','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN KEPONG','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN KUALA PAUH','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN PETALING','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN SALAK SOUTH','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='PEKAN SUNGAI PENCHALA','WP KUALA LUMPUR',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='LABUAN','WP LABUAN',mukim_district_state['state'])
mukim_district_state['state']=np.where(mukim_district_state['mukim']=='DONGGONGON','SABAH',mukim_district_state['state'])
mukim_district_state


,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [ ]:
#mukim_district_state['address_partial']=mukim_district_state['mukim']+','+mukim_district_state['district']+','+mukim_district_state['state']
#mukim_district_state

In [59]:
set(mukim_district_state['state'].to_list())

{'JOHOR',
 'KEDAH',
 'KELANTAN',
 'MELAKA',
 'NEGERI SEMBILAN',
 'PAHANG',
 'PERAK',
 'PERLIS',
 'PULAU PINANG',
 'SABAH',
 'SARAWAK',
 'SELANGOR',
 'TERENGGANU',
 'WP KUALA LUMPUR',
 'WP LABUAN',
 'WP PUTRAJAYA'}

- Read Data from Source 4 (Location, PostCode, District, State)

In [60]:
location=pd.read_csv('malaysia-postcodes-locations.csv',dtype=str,header=None)
location.columns=['postcode','location','district','state']
location

,postcode,location,district,state
0,01000,Alor Redeh,Kangar,PLS
1,01000,Alor Tok Terang,Kangar,PLS
2,01000,Bohor Gelam,Kangar,PLS
3,01000,Jalan Abi Tok Hashim,Kangar,PLS
4,01000,Jalan Alor Sena,Kangar,PLS
...,...,...,...,...
56229,98850,Sungai Seloating,Lawas,SRW
56230,98850,Trusan,Lawas,SRW
56231,98850,Tuaran,Lawas,SRW
56232,98857,Peti Surat,Lawas,SRW


In [61]:
set(location['state'].to_list())

{'JHR',
 'KDH',
 'KTN',
 'KUL',
 'LBN',
 'MLK',
 'NSN',
 'PHG',
 'PJY',
 'PLS',
 'PNG',
 'PRK',
 'SBH',
 'SGR',
 'SRW',
 'TRG'}

- DataFrame (location) Transformation to Standardize State as of (mukim_district_state)

In [62]:
location['state']=np.where(location['state']=='JHR','JOHOR',location['state'])
location['state']=np.where(location['state']=='KDH','KEDAH',location['state'])
location['state']=np.where(location['state']=='KTN', 'KELANTAN', location['state'])
location['state']=np.where(location['state']=='KUL', 'WP KUALA LUMPUR', location['state'])
location['state']=np.where(location['state']=='LBN', 'WP LABUAN', location['state'])
location['state']=np.where(location['state']=='MLK', 'MELAKA', location['state'])
location['state']=np.where(location['state']=='NSN', 'NEGERI SEMBILAN', location['state'])
location['state']=np.where(location['state']=='PHG', 'PAHANG', location['state'])
location['state']=np.where(location['state']=='PJY', 'WP PUTRAJAYA', location['state'])
location['state']=np.where(location['state']=='PLS', 'PERLIS', location['state'])
location['state']=np.where(location['state']=='PNG', 'PULAU PINANG', location['state'])
location['state']=np.where(location['state']=='PRK', 'PERAK', location['state'])
location['state']=np.where(location['state']=='SBH', 'SABAH', location['state'])
location['state']=np.where(location['state']=='SGR', 'SELANGOR', location['state'])
location['state']=np.where(location['state']=='SRW', 'SARAWAK', location['state'])
location['state']=np.where(location['state']=='TRG', 'TERENGGANU', location['state'])

location

,postcode,location,district,state
0,01000,Alor Redeh,Kangar,PERLIS
1,01000,Alor Tok Terang,Kangar,PERLIS
2,01000,Bohor Gelam,Kangar,PERLIS
3,01000,Jalan Abi Tok Hashim,Kangar,PERLIS
4,01000,Jalan Alor Sena,Kangar,PERLIS
...,...,...,...,...
56229,98850,Sungai Seloating,Lawas,SARAWAK
56230,98850,Trusan,Lawas,SARAWAK
56231,98850,Tuaran,Lawas,SARAWAK
56232,98857,Peti Surat,Lawas,SARAWAK


In [64]:
location['district']=location['district'].str.upper()
location

,postcode,location,district,state
0,01000,Alor Redeh,KANGAR,PERLIS
1,01000,Alor Tok Terang,KANGAR,PERLIS
2,01000,Bohor Gelam,KANGAR,PERLIS
3,01000,Jalan Abi Tok Hashim,KANGAR,PERLIS
4,01000,Jalan Alor Sena,KANGAR,PERLIS
...,...,...,...,...
56229,98850,Sungai Seloating,LAWAS,SARAWAK
56230,98850,Trusan,LAWAS,SARAWAK
56231,98850,Tuaran,LAWAS,SARAWAK
56232,98857,Peti Surat,LAWAS,SARAWAK


In [63]:
mukim_district_state['state']=mukim_district_state['state'].str.upper()
mukim_district_state['district']=mukim_district_state['district'].str.upper()
mukim_district_state['mukim']=mukim_district_state['mukim'].str.upper()
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [65]:
prep3=pd.concat([location, mukim_district_state.drop(columns='state_id')]).reset_index(drop=True)
prep3.to_csv('prep3.csv',index=False)
prep3

,postcode,location,district,state,mukim
0,01000,Alor Redeh,KANGAR,PERLIS,NaN
1,01000,Alor Tok Terang,KANGAR,PERLIS,NaN
2,01000,Bohor Gelam,KANGAR,PERLIS,NaN
3,01000,Jalan Abi Tok Hashim,KANGAR,PERLIS,NaN
4,01000,Jalan Alor Sena,KANGAR,PERLIS,NaN
...,...,...,...,...,...
58219,NaN,NaN,BINTULU,SARAWAK,ANAP
58220,NaN,NaN,BINTULU,SARAWAK,KAKUS
58221,NaN,NaN,BINTULU,SARAWAK,JELALONG
58222,NaN,NaN,BINTULU,SARAWAK,BINIA


In [66]:
mukim_district_state=pd.read_csv('prep3.csv',dtype='str')
mukim_district_state

,postcode,location,district,state,mukim
0,01000,Alor Redeh,KANGAR,PERLIS,NaN
1,01000,Alor Tok Terang,KANGAR,PERLIS,NaN
2,01000,Bohor Gelam,KANGAR,PERLIS,NaN
3,01000,Jalan Abi Tok Hashim,KANGAR,PERLIS,NaN
4,01000,Jalan Alor Sena,KANGAR,PERLIS,NaN
...,...,...,...,...,...
58219,NaN,NaN,BINTULU,SARAWAK,ANAP
58220,NaN,NaN,BINTULU,SARAWAK,KAKUS
58221,NaN,NaN,BINTULU,SARAWAK,JELALONG
58222,NaN,NaN,BINTULU,SARAWAK,BINIA


- Function to Return Postcode Value of Reference Table (location) if String Combination of Main Table (mukim_district_state) Exist 

In [ ]:
def find_state_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, district, location) for that particular postcode

def find_state_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, location) for that particular postcode

def find_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (district, location) for that particular postcode
 

In [ ]:
mukim_district_state['postcode1'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_district_location(x, location)).to_list()
mukim_district_state['postcode2'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_location(x, location)).to_list()
mukim_district_state['postcode3'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_district_location(x, location)).to_list()

- Split Postcode based on Combination of Scenario 1,2 and 3

In [ ]:
mukim_district_state['postcode1'] = mukim_district_state['postcode1'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode2'] = mukim_district_state['postcode2'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode3'] = mukim_district_state['postcode3'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state

- Extract Postcode Value by Priority (1,2,3)

In [ ]:
mukim_district_state['postcode']=mukim_district_state['postcode1']
mukim_district_state['postcode']=np.where(mukim_district_state['postcode']=='',mukim_district_state['postcode2'],mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where(mukim_district_state['postcode']=='',mukim_district_state['postcode3'],mukim_district_state['postcode'])
mukim_district_state

In [ ]:
#mukim_district_state.to_csv('prep1.csv',index=False)

In [ ]:
mukim_district_state=pd.read_csv('prep1.csv',dtype=str)
mukim_district_state
#mukim_district_state

- DataFrame (mukim_district_state) Transformation to Fill In Null Value for Postcode

In [67]:
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ABI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERSERI')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHUPING')&(mukim_district_state['state']=='PERLIS'), '02500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='UTAN AJI')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KECHOR')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG ANAI')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG BATANG')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NGULANG')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ORAN')&(mukim_district_state['state']=='PERLIS'), '02500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG SIDING')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PAYA')&(mukim_district_state['state']=='PERLIS'), '02400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGLANG')&(mukim_district_state['state']=='PERLIS'), '02800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENA')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERIAB')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI ADAM')&(mukim_district_state['state']=='PERLIS'), '02600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TITI TINGGI')&(mukim_district_state['state']=='PERLIS'), '02100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='WANG BINTONG')&(mukim_district_state['state']=='PERLIS'), '01000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERLIS')]


,postcode,location,district,state,mukim
56235,NaN,NaN,PERLIS,PERLIS,ARAU
56239,NaN,NaN,PERLIS,PERLIS,JEJAWI
56240,NaN,NaN,PERLIS,PERLIS,KAYANG
56242,NaN,NaN,PERLIS,PERLIS,KUALA PERLIS
56247,NaN,NaN,PERLIS,PERLIS,PADANG PAUH
56256,NaN,NaN,PERLIS,PERLIS,BANDAR ARAU
56257,NaN,NaN,PERLIS,PERLIS,BANDAR KANGAR
56258,NaN,NaN,PERLIS,PERLIS,PEKAN KUALA PERLIS
56259,NaN,NaN,PERLIS,PERLIS,PEKAN KAKI BUKIT


In [68]:
# show all row
# pd.set_option('display.max_rows', None)

# mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PULAU PINANG')]

# spt
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 17')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 18')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 19')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 20')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 21')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PRAI')&(mukim_district_state['district']=='SEBERANG PERAI TENGAH'), '13600', mukim_district_state['postcode'])

# spu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13110', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '12300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '12100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI UTARA'), '13800', mukim_district_state['postcode'])

# sps
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14320', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14110', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 15')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14120', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='SEBERANG PERAI SELATAN'), '14100', mukim_district_state['postcode'])

# tl
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 13')&(mukim_district_state['district']=='TIMOR LAUT'), '11500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 14')&(mukim_district_state['district']=='TIMOR LAUT'), '11060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 16')&(mukim_district_state['district']=='TIMOR LAUT'), '11500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 17')&(mukim_district_state['district']=='TIMOR LAUT'), '11100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 18')&(mukim_district_state['district']=='TIMOR LAUT'), '11200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU FERRINGGI')&(mukim_district_state['district']=='TIMOR LAUT'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GLUGOR')&(mukim_district_state['district']=='TIMOR LAUT'), '10150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GEORGE TOWN')&(mukim_district_state['district']=='TIMOR LAUT'), '10450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG BUNGAH')&(mukim_district_state['district']=='TIMOR LAUT'), '11200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG PINANG')&(mukim_district_state['district']=='TIMOR LAUT'), '10470', mukim_district_state['postcode'])

# bd
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 1')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 2')&(mukim_district_state['district']=='BARAT DAYA'), '11050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 3')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 4')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 5')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 6')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 7')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 8')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 9')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 10')&(mukim_district_state['district']=='BARAT DAYA'), '11900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 11')&(mukim_district_state['district']=='BARAT DAYA'), '11920', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM 12')&(mukim_district_state['district']=='BARAT DAYA'), '11900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM A')&(mukim_district_state['district']=='BARAT DAYA'), '11010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM B')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM C')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM D')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM E')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM F')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM G')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM H')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM I')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MUKIM J')&(mukim_district_state['district']=='BARAT DAYA'), '11000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PULAU PINANG')]


,postcode,location,district,state,mukim
56921,NaN,NaN,SEBERANG PERAI TENGAH,PULAU PINANG,BANDAR BUKIT MERTAJAM
56939,NaN,NaN,SEBERANG PERAI UTARA,PULAU PINANG,BANDAR BUTTERWORTH
56940,NaN,NaN,SEBERANG PERAI UTARA,PULAU PINANG,BANDAR KEPALA BATAS
56957,NaN,NaN,SEBERANG PERAI SELATAN,PULAU PINANG,BANDAR NIBONG TEBAL
56958,NaN,NaN,SEBERANG PERAI SELATAN,PULAU PINANG,BANDAR SUNGAI BAKAP
56964,NaN,NaN,TIMOR LAUT,PULAU PINANG,BANDAR AYER ITAM
56966,NaN,NaN,TIMOR LAUT,PULAU PINANG,BANDAR BUKIT BENDERA
56969,NaN,NaN,TIMOR LAUT,PULAU PINANG,BANDAR JELUTONG
56971,NaN,NaN,TIMOR LAUT,PULAU PINANG,BANDAR TANJONG TOKONG
56995,NaN,NaN,BARAT DAYA,PULAU PINANG,BANDAR BALIK PULAU


In [69]:
# kubang pasu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AH')&(mukim_district_state['district']=='KUBANG PASU'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BINJAL')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELONG')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT TINGGI')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HUSBA')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JERLUN')&(mukim_district_state['district']=='KUBANG PASU'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEPELU')&(mukim_district_state['district']=='KUBANG PASU'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUBANG PASU')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='NAGA')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG PERAHU')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERING')&(mukim_district_state['district']=='KUBANG PASU'), '06100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUTAT')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SANGLANG')&(mukim_district_state['district']=='KUBANG PASU'), '06150', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI LAKA')&(mukim_district_state['district']=='KUBANG PASU'), '06010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMIN')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='WANG TEPUS')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHANGLUN')&(mukim_district_state['district']=='KUBANG PASU'), '06010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BANDAR DARULAMAN')&(mukim_district_state['district']=='KUBANG PASU'), '06000', mukim_district_state['postcode'])


#padang terap
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG TUNGGANG KANAN')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG TUNGGANG KIRI')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELIMBING KANAN')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELIMBING KIRI')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KURONG HITAM')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERNAK')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERAP KANAN')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TERAP KIRI')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEKAI')&(mukim_district_state['district']=='PADANG TERAP'), '06400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TOLAK')&(mukim_district_state['district']=='PADANG TERAP'), '06300', mukim_district_state['postcode'])

#kuala muda 
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT MERIAM')&(mukim_district_state['district']=='KUALA MUDA'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA')&(mukim_district_state['district']=='KUALA MUDA'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PEKULA')&(mukim_district_state['district']=='KUALA MUDA'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEMELING')&(mukim_district_state['district']=='KUALA MUDA'), '06000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOI KIRI')&(mukim_district_state['district']=='KUALA MUDA'), '06000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KEDAH')]

,postcode,location,district,state,mukim
56260,NaN,NaN,KOTA SETAR,KEDAH,ALOR MALAI
56261,NaN,NaN,KOTA SETAR,KEDAH,ALOR MERAH
56262,NaN,NaN,KOTA SETAR,KEDAH,ANAK BUKIT
56263,NaN,NaN,KOTA SETAR,KEDAH,BUKIT PINANG
56264,NaN,NaN,KOTA SETAR,KEDAH,DERGA
...,...,...,...,...,...
57981,NaN,NaN,PADANG TERAP,KEDAH,NaN
57982,NaN,NaN,PADANG TERAP,KEDAH,NaN
57983,NaN,NaN,PADANG TERAP,KEDAH,NaN
57984,NaN,NaN,PADANG TERAP,KEDAH,NaN


In [70]:
#print(len(mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')]))
#pd.set_option('display.max_rows', None)
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')]

# batang padang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATANG PADANG')&(mukim_district_state['district']=='BATANG PADANG'), '35000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SLIM')&(mukim_district_state['district']=='BATANG PADANG'), '35800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHENDERIAN')&(mukim_district_state['district']=='BATANG PADANG'), '35350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG MALIM')&(mukim_district_state['district']=='BATANG PADANG'), '35600', mukim_district_state['postcode'])

# manjung
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERUAS')&(mukim_district_state['district']=='MANJUNG'), '32700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGKALAN BAHARU')&(mukim_district_state['district']=='MANJUNG'), '34900', mukim_district_state['postcode'])

# kinta
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELANJA')&(mukim_district_state['district']=='KINTA'), '32610', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG TUALANG')&(mukim_district_state['district']=='KINTA'), '31800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR IPOH (U)')&(mukim_district_state['district']=='KINTA'), '30020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR IPOH (S)')&(mukim_district_state['district']=='KINTA'), '31350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MENGELEMBU')&(mukim_district_state['district']=='KINTA'), '31450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PAPAN')&(mukim_district_state['district']=='KINTA'), '31550', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEPUTEH')&(mukim_district_state['district']=='KINTA'), '31560', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI RAYA')&(mukim_district_state['district']=='KINTA'), '31300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG RAMBUTAN')&(mukim_district_state['district']=='KINTA'), '31250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERONOH')&(mukim_district_state['district']=='KINTA'), '31750', mukim_district_state['postcode'])

# kerian
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERIAH')&(mukim_district_state['district']=='KERIAN'), '34300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUNONG SEMANGGOL')&(mukim_district_state['district']=='KERIAN'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SELINSING')&(mukim_district_state['district']=='KERIAN'), '34400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG PIANDANG')&(mukim_district_state['district']=='KERIAN'), '34350', mukim_district_state['postcode'])

# hilir perak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BAGAN DATUK')&(mukim_district_state['district']=='HILIR PERAK'), '36100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHANGKAT JONG')&(mukim_district_state['district']=='HILIR PERAK'), '36700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RUNGKUP')&(mukim_district_state['district']=='HILIR PERAK'), '36200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI MANIK')&(mukim_district_state['district']=='HILIR PERAK'), '36750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK BARU')&(mukim_district_state['district']=='HILIR PERAK'), '36300', mukim_district_state['postcode'])

# hulu perak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELUKAR SEMANG')&(mukim_district_state['district']=='HULU PERAK'), '33200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELUM')&(mukim_district_state['district']=='HULU PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DURIAN PIPIT')&(mukim_district_state['district']=='HULU PERAK'), '33030', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GRIK')&(mukim_district_state['district']=='HULU PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERUNAI')&(mukim_district_state['district']=='HULU PERAK'), '33310', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENERING')&(mukim_district_state['district']=='HULU PERAK'), '33410', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMELONG')&(mukim_district_state['district']=='HULU PERAK'), '33400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMENGOR')&(mukim_district_state['district']=='HULU PERAK'), '33300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR LAWN')&(mukim_district_state['district']=='HULU PERAK'), '33410', mukim_district_state['postcode'])

# perak tengah
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR')&(mukim_district_state['district']=='PERAK TENGAH'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BELANJA')&(mukim_district_state['district']=='PERAK TENGAH'), '32800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JAYA BARU')&(mukim_district_state['district']=='PERAK TENGAH'), '32000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KAMPONG GAJAH')&(mukim_district_state['district']=='PERAK TENGAH'), '36800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LAMBOR KIRI')&(mukim_district_state['district']=='PERAK TENGAH'), '32600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR PANJANG HULU')&(mukim_district_state['district']=='PERAK TENGAH'), '36800', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')]


,postcode,location,district,state,mukim
56998,NaN,NaN,BATANG PADANG,PERAK,BIDOR
56999,NaN,NaN,BATANG PADANG,PERAK,CHENDERIANG
57000,NaN,NaN,BATANG PADANG,PERAK,HULU BERNAM BARAT
57001,NaN,NaN,BATANG PADANG,PERAK,HULU BERNAM TIMOR
57003,NaN,NaN,BATANG PADANG,PERAK,SUNGKAI
...,...,...,...,...,...
57180,NaN,NaN,KAMPAR,PERAK,TEJA
57181,NaN,NaN,KAMPAR,PERAK,BANDAR GOPENG
57182,NaN,NaN,KAMPAR,PERAK,BANDAR KAMPAR
57183,NaN,NaN,KAMPAR,PERAK,PEKAN KOTA BAHARU


In [71]:
# print(len(mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]))
# mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]

# klang
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KLANG')&(mukim_district_state['district']=='KLANG'), '41400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KLANG')&(mukim_district_state['district']=='KLANG'), '41000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PORT SWETTENHAM')&(mukim_district_state['district']=='KLANG'), '42000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SHAH ALAM')&(mukim_district_state['district']=='KLANG'), '40000', mukim_district_state['postcode'])

# kuala langat
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR')&(mukim_district_state['district']=='KUALA LANGAT'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JUGRA')&(mukim_district_state['district']=='KUALA LANGAT'), '42960', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG DUABELAS')&(mukim_district_state['district']=='KUALA LANGAT'), '42700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TANJONG SEPAT')&(mukim_district_state['district']=='KUALA LANGAT'), '42800', mukim_district_state['postcode'])

# ulu langat
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR CHERAS')&(mukim_district_state['district']=='ULU LANGAT'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU LANGAT')&(mukim_district_state['district']=='ULU LANGAT'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALAKONG')&(mukim_district_state['district']=='ULU LANGAT'), '43300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 9, CHERAS')&(mukim_district_state['district']=='ULU LANGAT'), '43200', mukim_district_state['postcode'])

# ulu selangor
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BULUH TELOR')&(mukim_district_state['district']=='ULU SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU YAM')&(mukim_district_state['district']=='ULU SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PERETAK')&(mukim_district_state['district']=='ULU SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI GUMUT')&(mukim_district_state['district']=='ULU SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SUNGAI TINGGI')&(mukim_district_state['district']=='ULU SELANGOR'), '44200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU YAM')&(mukim_district_state['district']=='ULU SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU YAM BAHARU')&(mukim_district_state['district']=='ULU SELANGOR'), '44300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KUBU BAHARU')&(mukim_district_state['district']=='ULU SELANGOR'), '44000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI CHIK')&(mukim_district_state['district']=='ULU SELANGOR'), '44300', mukim_district_state['postcode'])

# petaling
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SHAH ALAM')&(mukim_district_state['district']=='PETALING'), '40250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR DAMANSARA')&(mukim_district_state['district']=='PETALING'), '46350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GLENMARIE')&(mukim_district_state['district']=='PETALING'), '40250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA SELATAN')&(mukim_district_state['district']=='PETALING'), '46000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SRI DAMANSARA')&(mukim_district_state['district']=='PETALING'), '52200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUKIT RAJA')&(mukim_district_state['district']=='PETALING'), '40170', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='DAMANSARA')&(mukim_district_state['district']=='PETALING'), '52200', mukim_district_state['postcode'])

# gombak
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GOMBAK SETIA')&(mukim_district_state['district']=='GOMBAK'), '53100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ULU KELANG')&(mukim_district_state['district']=='GOMBAK'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KEPONG')&(mukim_district_state['district']=='GOMBAK'), '52100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUNDANG')&(mukim_district_state['district']=='GOMBAK'), '48050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI PUSU')&(mukim_district_state['district']=='GOMBAK'), '53000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]


,postcode,location,district,state,mukim
57311,NaN,NaN,KLANG,SELANGOR,KAPAR
57315,NaN,NaN,KLANG,SELANGOR,BANDAR SULTAN SULAIMAN
57317,NaN,NaN,KLANG,SELANGOR,PEKAN BUKIT KEMUNING
57318,NaN,NaN,KLANG,SELANGOR,PEKAN KAPAR
57319,NaN,NaN,KLANG,SELANGOR,PEKAN MERU
...,...,...,...,...,...
57523,NaN,NaN,SEPANG,SELANGOR,PEKAN BUKIT PRANG
57524,NaN,NaN,SEPANG,SELANGOR,PEKAN DATO BAKAR BAGINDA
57525,NaN,NaN,SEPANG,SELANGOR,PEKAN BARU SALAK TINGGI
57526,NaN,NaN,SEPANG,SELANGOR,PEKAN SUNGAI MERAB


In [72]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='WP KUALA LUMPUR')]

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU')&(mukim_district_state['district']=='KUALA LANGAT'), '42800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHERAS')&(mukim_district_state['district']=='ULU LANGAT'), '43200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PETALING')&(mukim_district_state['district']=='PETALING'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA')&(mukim_district_state['district']=='PETALING'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KELANG')&(mukim_district_state['district']=='GOMBAK'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETAPAK')&(mukim_district_state['district']=='GOMBAK'), '53100', mukim_district_state['postcode'])

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '52100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHERAS')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '56000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PETALING')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46675', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETAPAK')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '53300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA LUMPUR')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PETALING JAYA')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '46200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KELANG')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '68000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA LUMPUR')&(mukim_district_state['state']=='WP KUALA LUMPUR'), '50450', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='WP KUALA LUMPUR')]

,postcode,location,district,state,mukim
57335,NaN,NaN,KUALA LANGAT,WP KUALA LUMPUR,PEKAN BATU
57399,NaN,NaN,ULU LANGAT,WP KUALA LUMPUR,AMPANG
57477,NaN,NaN,PETALING,WP KUALA LUMPUR,PEKAN SUNGAI PENCHALA
57528,NaN,NaN,NaN,WP KUALA LUMPUR,AMPANG
57537,NaN,NaN,NaN,WP KUALA LUMPUR,BANDAR BANDAR BAHARU SUNGAI BESI
57538,NaN,NaN,NaN,WP KUALA LUMPUR,PEKAN BATU
57539,NaN,NaN,NaN,WP KUALA LUMPUR,PEKAN BATU CAVES
57540,NaN,NaN,NaN,WP KUALA LUMPUR,PEKAN KEPONG
57541,NaN,NaN,NaN,WP KUALA LUMPUR,PEKAN KUALA PAUH
57542,NaN,NaN,NaN,WP KUALA LUMPUR,PEKAN PETALING


In [73]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')]

# kuching
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUEH LAND')&(mukim_district_state['district']=='KUCHING'), '94100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='STUNGKOR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SAMPADI')&(mukim_district_state['district']=='KUCHING'), '93070', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JAGOI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENGGI POAK')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SALAK')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PANGKALAN AMPAT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUCHING UTARA')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUCHING TENGAH')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUCHING')&(mukim_district_state['district']=='KUCHING'), '93100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SENTAH-SEGU')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU KAWA')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI TENGAH')&(mukim_district_state['district']=='KUCHING'), '93050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI KITANG')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU KITANG')&(mukim_district_state['district']=='KUCHING'), '93250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU 15, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='175, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 19, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BATU 24, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANGKALAN KUT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BELIONG')&(mukim_district_state['district']=='KUCHING'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BAKO')&(mukim_district_state['district']=='KUCHING'), '93010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JANGKAR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR RAMBUNGAN')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR STUNKOR')&(mukim_district_state['district']=='KUCHING'), '94500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KRANJI')&(mukim_district_state['district']=='KUCHING'), '94750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SINIWAN')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MUSI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAI TON')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BIDI')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PANGKALAN TEBANG')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PEJIRU')&(mukim_district_state['district']=='KUCHING'), '94000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BARATOK')&(mukim_district_state['district']=='KUCHING'), '94200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERBAT')&(mukim_district_state['district']=='KUCHING'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 17')&(mukim_district_state['district']=='KUCHING'), '95000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JALAN BATU 17, JALAN SENGGANG')&(mukim_district_state['district']=='KUCHING'), '95000', mukim_district_state['postcode'])

# samarahan
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MENUKU')&(mukim_district_state['district']=='SAMARAHAN'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SAMARAHAN')&(mukim_district_state['district']=='SAMARAHAN'), '94300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MELIKIN LAND')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PUNDA-SABAL')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI MERAH')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNAGAI MERANG')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SUNGAI PALAH')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEDEBU')&(mukim_district_state['district']=='SAMARAHAN'), '94760', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR BALAI RINGIN')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAMBIRAT')&(mukim_district_state['district']=='SAMARAHAN'), '94600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SIMUJAN')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TEBELU')&(mukim_district_state['district']=='SAMARAHAN'), '94850', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TERBAT')&(mukim_district_state['district']=='SAMARAHAN'), '94700', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')]


,postcode,location,district,state,mukim
57995,NaN,NaN,KUCHING,SARAWAK,GADING LUNDU
58000,NaN,NaN,KUCHING,SARAWAK,MATANG
58007,NaN,NaN,KUCHING,SARAWAK,MUARA TEBAS
58009,NaN,NaN,KUCHING,SARAWAK,"BATU 8, JALAN MATANG"
58019,NaN,NaN,KUCHING,SARAWAK,BANDAR SEMATANG
...,...,...,...,...,...
58219,NaN,NaN,BINTULU,SARAWAK,ANAP
58220,NaN,NaN,BINTULU,SARAWAK,KAKUS
58221,NaN,NaN,BINTULU,SARAWAK,JELALONG
58222,NaN,NaN,BINTULU,SARAWAK,BINIA


In [74]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KELANTAN')]

mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ENGGONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GAJAH')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KANDEK')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENOR')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA GERIS')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KRAI')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA NAL')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA PERGAU')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA STONG')&(mukim_district_state['district']=='KUALA KRAI'), '18200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MAMBONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='MANJOR')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELEKONG')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KRAI')&(mukim_district_state['district']=='KUALA KRAI'), '18000', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KELANTAN')]

,postcode,location,district,state,mukim
56485,NaN,NaN,BACHOK,KELANTAN,ALOR BAKAT
56486,NaN,NaN,BACHOK,KELANTAN,BACKHOK
56487,NaN,NaN,BACHOK,KELANTAN,BATOR
56488,NaN,NaN,BACHOK,KELANTAN,CHAP
56489,NaN,NaN,BACHOK,KELANTAN,CHERANG HANGUS
...,...,...,...,...,...
56799,NaN,NaN,KECIL LOJING,KELANTAN,BLAU
56800,NaN,NaN,KECIL LOJING,KELANTAN,HAU
56801,NaN,NaN,KECIL LOJING,KELANTAN,HENDROP
56802,NaN,NaN,KECIL LOJING,KELANTAN,SIGAR


In [75]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='MELAKA')]
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ALAI')&(mukim_district_state['district']=='MELAKA TENGAH'), '75460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BACHANG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75350', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERTAM')&(mukim_district_state['district']=='MELAKA TENGAH'), '76450', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHENG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75250', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KRUBONG')&(mukim_district_state['district']=='MELAKA TENGAH'), '75260', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PADANG TEMU')&(mukim_district_state['district']=='MELAKA TENGAH'), '75050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PRINGGIT')&(mukim_district_state['district']=='MELAKA TENGAH'), '75400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANGGA BATU')&(mukim_district_state['district']=='MELAKA TENGAH'), '76400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG KELING')&(mukim_district_state['district']=='MELAKA TENGAH'), '76400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANJONG MINYAK')&(mukim_district_state['district']=='MELAKA TENGAH'), '75260', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELOK MAS')&(mukim_district_state['district']=='MELAKA TENGAH'), '75460', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR MELAKA')&(mukim_district_state['district']=='MELAKA TENGAH'), '75400', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='MELAKA')]


,postcode,location,district,state,mukim
57707,NaN,NaN,MELAKA TENGAH,MELAKA,AYER MOLEK
57709,NaN,NaN,MELAKA TENGAH,MELAKA,BALAI PANJANG
57710,NaN,NaN,MELAKA TENGAH,MELAKA,BATU BERENDAM
57712,NaN,NaN,MELAKA TENGAH,MELAKA,BUKIT BARU
57713,NaN,NaN,MELAKA TENGAH,MELAKA,BUKIT KATIL
...,...,...,...,...,...
57810,NaN,NaN,ALOR GAJAH,MELAKA,BANDAR PULAU SEBANG
57811,NaN,NaN,ALOR GAJAH,MELAKA,PEKAN DURIAN TUNGGAL
57812,NaN,NaN,ALOR GAJAH,MELAKA,PEKAN KUALA SUNGAI BARU
57813,NaN,NaN,ALOR GAJAH,MELAKA,PEKAN LUBOK CHINA


In [76]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')]

#jelebu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELAMI LEMI')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KENABOI')&(mukim_district_state['district']=='JELEBU'), '71650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KELAWANG')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERIANG HILIR')&(mukim_district_state['district']=='JELEBU'), '72400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA KELAWANG')&(mukim_district_state['district']=='JELEBU'), '71600', mukim_district_state['postcode'])

#seremban
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='LABU')&(mukim_district_state['district']=='SEREMBAN'), '71900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SEREMBAN')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SETUL')&(mukim_district_state['district']=='SEREMBAN'), '71800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEREMBAN')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SEREMBAN UTAMA')&(mukim_district_state['district']=='SEREMBAN'), '70000', mukim_district_state['postcode'])

#tampin
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='AYER KUNING')&(mukim_district_state['district']=='TAMPIN'), '73200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GEMAS')&(mukim_district_state['district']=='TAMPIN'), '73400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERU')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='REPAH')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBONG')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR GEMAS')&(mukim_district_state['district']=='TAMPIN'), '73400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TAMPIN')&(mukim_district_state['district']=='TAMPIN'), '73000', mukim_district_state['postcode'])

#jempol
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JELAI')&(mukim_district_state['district']=='JEMPOL'), '73500', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ROMPIN')&(mukim_district_state['district']=='JEMPOL'), '73430', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR SERTING')&(mukim_district_state['district']=='JEMPOL'), '72100', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')]


,postcode,location,district,state,mukim
57549,NaN,NaN,JELEBU,NEGERI SEMBILAN,PERADONG
57550,NaN,NaN,JELEBU,NEGERI SEMBILAN,PERTANG
57553,NaN,NaN,JELEBU,NEGERI SEMBILAN,PEKAN KUALA KELAWANG
57554,NaN,NaN,JELEBU,NEGERI SEMBILAN,PEKAN PERTANG
57555,NaN,NaN,JELEBU,NEGERI SEMBILAN,PEKAN TITI
...,...,...,...,...,...
57701,NaN,NaN,JEMPOL,NEGERI SEMBILAN,PEKAN KUALA JELAI
57702,NaN,NaN,JEMPOL,NEGERI SEMBILAN,PEKAN ROMPIN
57703,NaN,NaN,JEMPOL,NEGERI SEMBILAN,PEKAN LADANG GEDDES
57704,NaN,NaN,JEMPOL,NEGERI SEMBILAN,PEKAN MAHSAN


In [77]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')]
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PLENTONG')&(mukim_district_state['district']=='JOHOR BAHRU'), '81750', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAI')&(mukim_district_state['district']=='JOHOR BAHRU'), '81300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEMBRAU')&(mukim_district_state['district']=='JOHOR BAHRU'), '81100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR JOHOR BAHRU')&(mukim_district_state['district']=='JOHOR BAHRU'), '80000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUNDANG')&(mukim_district_state['district']=='LEDANG'), '84600', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BADNAR BUKIT KANGKAR')&(mukim_district_state['district']=='LEDANG'), '84300', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')]

,postcode,location,district,state,mukim
57815,NaN,NaN,BATU PAHAT,JOHOR,BAGAN
57816,NaN,NaN,BATU PAHAT,JOHOR,CHAAH BAHRU
57817,NaN,NaN,BATU PAHAT,JOHOR,KAMPONG BAHRU
57818,NaN,NaN,BATU PAHAT,JOHOR,LINAU
57819,NaN,NaN,BATU PAHAT,JOHOR,LUBOK
...,...,...,...,...,...
57942,NaN,NaN,LEDANG,JOHOR,BANDAR PARIT BUNGA
57943,NaN,NaN,LEDANG,JOHOR,BANDAR SEROM
57944,NaN,NaN,LEDANG,JOHOR,BANDAR SUNGAI MATI
57945,NaN,NaN,LEDANG,JOHOR,BANDAR TANGKAK


In [78]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')]

#kuala terengganu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGGUL')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU JABUR')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR SEMUT')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBAK')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELUK KALUNG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '24100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU BURUK')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU RAKIT')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21020', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CABANG TIGA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CENERIANG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21080', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELUGUR KEDAI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GELUGUR RAJA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KEPUNG')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA IBAI')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20400', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA NERUS')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21300', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PALUH')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGADANG BULUH')&(mukim_district_state['district']=='KUALA TERENGGANU'), '21070', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PULAU-PULAU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='RENGAS')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='SERADA')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TOK JAMAL')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20050', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUALA TERENGGANU')&(mukim_district_state['district']=='KUALA TERENGGANU'), '20000', mukim_district_state['postcode'])

#hulu terengganu
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU BERANG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TELEMUNG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TERENGGANU')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='JENAGUR')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA TELEMUNG')&(mukim_district_state['district']=='HULU TERENGGANU'), '21210', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENGHULU DIMAN')&(mukim_district_state['district']=='HULU TERENGGANU'), '21010', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TANGGUL')&(mukim_district_state['district']=='HULU TERENGGANU'), '21040', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TERSAT')&(mukim_district_state['district']=='HULU TERENGGANU'), '21700', mukim_district_state['postcode'])

#kemaman
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDI')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANGGUL')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU JABUR')&(mukim_district_state['district']=='KEMAMAN'), '24060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PASIR SEMUT')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TEBAK')&(mukim_district_state['district']=='KEMAMAN'), '24000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELUK KALUNG')&(mukim_district_state['district']=='KEMAMAN'), '24100', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')]

,postcode,location,district,state,mukim
56804,NaN,NaN,BESUT,TERENGGANU,BUKIT KENAK
56805,NaN,NaN,BESUT,TERENGGANU,BUKIT PUTERI
56806,NaN,NaN,BESUT,TERENGGANU,HULU BESUT
56807,NaN,NaN,BESUT,TERENGGANU,JABI
56808,NaN,NaN,BESUT,TERENGGANU,KAMPONG RAJA
...,...,...,...,...,...
56895,NaN,NaN,SETIU,TERENGGANU,CALUK
56896,NaN,NaN,SETIU,TERENGGANU,GUNTUNG
56897,NaN,NaN,SETIU,TERENGGANU,HULU NERUS
56898,NaN,NaN,SETIU,TERENGGANU,MERANG


In [79]:
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')]
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='HULU TELOM')&(mukim_district_state['district']=='CAMERON HIGHLANDS'), '39000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU LEPAR')&(mukim_district_state['district']=='KUANTAN'), '26330', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '26200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KUALA KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '25100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='PENOR')&(mukim_district_state['district']=='KUANTAN'), '26060', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR KUANTAN')&(mukim_district_state['district']=='KUANTAN'), '25000', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BATU YON')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BUDU')&(mukim_district_state['district']=='LIPIS'), '27310', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='CHEKA')&(mukim_district_state['district']=='LIPIS'), '27200', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='GUA')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ULU JELAI')&(mukim_district_state['district']=='LIPIS'), '27650', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KECHAU')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TELANG')&(mukim_district_state['district']=='LIPIS'), '27100', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='KERATONG')&(mukim_district_state['district']=='ROMPIN'), '26900', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='ROMPIN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='TIOMAN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN I')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN II')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN III')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR ROMPIN IV')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR TIOMAN')&(mukim_district_state['district']=='ROMPIN'), '26800', mukim_district_state['postcode'])
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BERA')&(mukim_district_state['district']=='BERA'), '28200', mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')]


,postcode,location,district,state,mukim
57185,NaN,NaN,BENTONG,PAHANG,BENTONG
57186,NaN,NaN,BENTONG,PAHANG,SABAI
57187,NaN,NaN,BENTONG,PAHANG,PELANGAI
57188,NaN,NaN,BENTONG,PAHANG,BANDAR BENTONG
57189,NaN,NaN,BENTONG,PAHANG,BANDAR KARAK
...,...,...,...,...,...
57306,NaN,NaN,BERA,PAHANG,TRIANG
57307,NaN,NaN,BERA,PAHANG,BANDAR TRIANG
57308,NaN,NaN,BERA,PAHANG,PEKAN DURIAN TAWAR
57309,NaN,NaN,BERA,PAHANG,PEKAN MENGKARAK


In [ ]:
mukim_district_state['postcode']=np.where((mukim_district_state['postcode'].isnull())&(mukim_district_state['mukim']=='BANDAR PUTRAJAYA')&(mukim_district_state['district']=='W. P. PUTRAJAYA'), '62100', mukim_district_state['postcode'])

In [ ]:
mukim_district_state=mukim_district_state[~((mukim_district_state['mukim'].isnull())&(mukim_district_state['district']=='PADANG TERAP'))]
mukim_district_state.to_csv('prep2.csv',index=False)

- Check if No Null Value for Postcode

In [ ]:
mukim_district_state[(mukim_district_state['postcode'].isnull())]

In [ ]:
mukim_district_state.head(5)

- Left Join (mukim_district_state, postcode_ref) on (postcode, district, state) columns

In [ ]:
postcode_ref=pd.read_csv('malaysia-postcodes-locations.csv', encoding='ISO-8859-1',dtype=str)
postcode_ref.columns = ['postcode', 'location', 'district', 'state']

postcode_ref['state']=np.where(postcode_ref['state']=='JHR','JOHOR',postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='KDH','KEDAH',postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='KTN', 'KELANTAN', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='KUL', 'WP KUALA LUMPUR', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='LBN', 'WP LABUAN', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='MLK', 'MELAKA', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='NSN', 'NEGERI SEMBILAN', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='PHG', 'PAHANG', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='PJY', 'WP PUTRAJAYA', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='PLS', 'PERLIS', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='PNG', 'PULAU PINANG', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='PRK', 'PERAK', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='SBH', 'SABAH', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='SGR', 'SELANGOR', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='SRW', 'SARAWAK', postcode_ref['state'])
postcode_ref['state']=np.where(postcode_ref['state']=='TRG', 'TERENGGANU', postcode_ref['state'])

postcode_ref['district']=postcode_ref['district'].str.upper()

postcode_ref.head()

In [ ]:
postcode_ref.info()

In [ ]:
mukim_district_state=pd.read_csv('prep2.csv',dtype=str)
mukim_district_state['state']=mukim_district_state['state'].str.strip()
mukim_district_state[mukim_district_state['mukim'].str.contains('JELUTONG',na=False)]

In [ ]:
list_postcode_ref=postcode_ref['district'].unique().tolist()
list_mukim_district_state=mukim_district_state['district'].unique().tolist()
list_mukim_district_state

In [ ]:
list_mukim_district_state2=mukim_district_state['mukim'].unique().tolist()
list_mukim_district_state2

In [ ]:
# list on item not in list
list_diff = [x for x in list_mukim_district_state if x not in list_postcode_ref]
list_diff

In [ ]:
district_additional=pd.DataFrame()
for district in list_diff:
    district_additional=pd.concat([district_additional,(mukim_district_state[(mukim_district_state['district']==district)][['postcode','mukim','district','state']].drop_duplicates())])

district_additional.rename(columns={'mukim':'location'}, inplace=True)
district_additional=district_additional.reset_index(drop=True)

In [ ]:
district_additional

In [ ]:
# update postcode_ref with district_additional
postcode_ref=pd.concat([postcode_ref,district_additional],ignore_index=True)
postcode_ref=postcode_ref.drop_duplicates()
postcode_ref=postcode_ref.reset_index(drop=True)

In [ ]:
len(pd.merge(postcode_ref,mukim_district_state,how='left',left_on=['postcode','district','state'],right_on=['postcode','district','state']))

In [ ]:
location_mukim_district_state=pd.merge(postcode_ref,mukim_district_state,how='left',left_on=['postcode','district','state'],right_on=['postcode','district','state'])
location_mukim_district_state.head(3) 

In [ ]:
location_mukim_district_state=location_mukim_district_state.drop_duplicates().reset_index(drop=True)

In [ ]:
len(location_mukim_district_state)

In [ ]:
location_mukim_district_state.drop(columns=['state_id', 'postcode1', 'postcode2', 'postcode3'], inplace=True)

In [ ]:
location_mukim_district_state['mukim']=location_mukim_district_state['mukim'].fillna('')

In [ ]:
location_mukim_district_state[location_mukim_district_state['mukim']=='MUKIM A']

In [ ]:
location_mukim_district_state['address']=location_mukim_district_state['location'].str.lower()+', '+location_mukim_district_state['mukim'].str.lower()+', '+location_mukim_district_state['district'].str.lower()+', '+location_mukim_district_state['postcode'].astype(str)+', '+location_mukim_district_state['state'].str.lower()
location_mukim_district_state

In [ ]:
location_mukim_district_state['address'] = location_mukim_district_state['address'].str.replace(r', , ', ', ', regex=True)
location_mukim_district_state

In [ ]:
location_mukim_district_state.to_csv('malaysia-postcodes-location-mukim-district-state.csv', index=False)